In [ ]:
import plotly.express as px
import pandas as pd
import seaborn as sns
import numpy as np
import plotly
import sweetviz as sv
import scipy
px.defaults.template = 'seaborn'
px.defaults.width = 700
px.defaults.height = 500
pd.set_option('display.max_columns', 30)

In [ ]:
df = pd.read_csv('Automobile price data _Raw_.csv')
df

In [ ]:
df.dtypes.value_counts()

In [ ]:
my_report = sv.analyze(df)
my_report.show_html()

In [ ]:
df.isna().sum().sum()

In [ ]:
px.histogram(df, x='price')

In [ ]:
from scipy.stats import kurtosis, skew
price = pd.to_numeric(df['price'], errors='coerce').dropna()
print('skew: ', round(skew(price), 2))
print('kurtosis', round(kurtosis(price), 2))

In [ ]:
df.dtypes

In [ ]:
df['normalized-losses'].value_counts()

In [ ]:
pd.to_numeric(df['normalized-losses'], 'coerce').dropna().plot.box()

In [ ]:
df['normalized-losses'].replace('?', np.nan, inplace=True)
df['normalized-losses'].fillna(df['normalized-losses'].median(), inplace=True)
df['normalized-losses'] = df['normalized-losses'].astype('int')

In [ ]:
df['make'].value_counts()

In [ ]:
df[df.columns[3]].value_counts()

In [ ]:
df[df.columns[4]].value_counts()

In [ ]:
df[df.columns[5]].value_counts()

In [ ]:
df.replace({'num-of-doors': '?'}, np.nan, inplace=True)
df.dropna(subset=['num-of-doors'], inplace=True)
df['num-of-doors'] = df['num-of-doors'].replace(['four', 'two'], [4, 2])
df['num-of-doors'].dtype

In [ ]:
df[df.columns[5]].value_counts()

In [ ]:
len(df)

In [ ]:
df[df.columns[6]].value_counts()

In [ ]:
df[df.columns[7]].value_counts()

In [ ]:
df[df.columns[8]].value_counts()

In [ ]:
df[df.columns[9]]

In [ ]:
df[df.columns[9]].dtype

In [ ]:
df[df.columns[10]].value_counts()

In [ ]:
df[df.columns[10]].dtype

In [ ]:
df[df.columns[11]].value_counts()

In [ ]:
df[df.columns[11]].dtype

In [ ]:
df[df.columns[12]].value_counts()

In [ ]:
df[df.columns[13]].dtype

In [ ]:
df[df.columns[14]].value_counts()

In [ ]:
df[df.columns[15]].value_counts()

In [ ]:
df['num-of-cylinders'] = df['num-of-cylinders'].replace(['four', 'six', 'five', 'eight', 'two', 'twelve', 'three'],
                                                        [4, 6, 5, 8, 2, 12, 3])

In [ ]:
df['num-of-cylinders'].value_counts()

In [ ]:
df[df.columns[16]].dtypes

In [ ]:
df[df.columns[17]].value_counts()

In [ ]:
df[df.columns[18]].value_counts()

In [ ]:
filt = df['bore'] != '?'
df = df[filt]
df['bore'] = df['bore'].astype('float')

In [ ]:
len(df)

In [ ]:
df[df.columns[19]].value_counts()

In [ ]:
df['stroke'] = pd.to_numeric(df['stroke'])

In [ ]:
df['stroke'].dtype

In [ ]:
df[df.columns[20]].dtype

In [ ]:
df[df.columns[21]].value_counts()

In [ ]:
df['horsepower'] = df['horsepower'].replace('?', np.nan)
df.dropna(subset=['horsepower'], inplace=True)
df['horsepower'] = pd.to_numeric(df['horsepower'])

In [ ]:
len(df)

In [ ]:
df['horsepower'].dtype

In [ ]:
df[df.columns[22]].value_counts()

In [ ]:
df['peak-rpm'] = df['peak-rpm'].astype('float')

In [ ]:
df['peak-rpm'].dtype

In [ ]:
df[df.columns[23]].value_counts()

In [ ]:
df[df.columns[23]].dtype

In [ ]:
df[df.columns[24]].dtype

In [ ]:
df[df.columns[25]].value_counts()

In [ ]:
df = df[df['price'] != '?']

In [ ]:
df['price'] = df['price'].astype('float')

In [ ]:
df

In [ ]:
df.select_dtypes('object')

In [ ]:
df.select_dtypes(exclude='object')

In [ ]:
df.dtypes.value_counts()

In [ ]:
encoded = pd.get_dummies(df.select_dtypes('object'), prefix=df.select_dtypes('object').columns)
encoded.head(2)

In [ ]:
df_ml = pd.concat([df.select_dtypes(exclude='object'), encoded], axis=1)
df_ml

In [ ]:
value_counts = df.apply(lambda x: len(x.value_counts()))
df_value_counts = pd.DataFrame([value_counts, df.dtypes]).T
df_value_counts.columns = ['value', 'dtype']
df_value_counts = df_value_counts.sort_values('value')
px.bar(df_value_counts,x=df_value_counts.index, color='dtype', y='value')

In [ ]:
dtypes = df.dtypes
dtypes

In [ ]:
cat_columns = list(dtypes[((dtypes == 'object') | (value_counts < 10))].index)
cat_columns

In [ ]:
num_columns = [i for i in df.columns if i not in cat_columns]
num_columns.remove('price')

In [ ]:
p_values = {}
for col in cat_columns:
    prices = []
    for group_name, df_group in df.groupby(col):
        prices.append(df_group['price'].values)
    f_static , p_value = scipy.stats.f_oneway(*prices)
    p_values[col] = p_value

In [ ]:
p_values = pd.Series(p_values).sort_values().astype('category')
px.bar(p_values, log_y=True, color=p_values.index)

In [ ]:
for feature in list(p_values.index[:4]):
    fig = px.histogram(df, x='price', color=feature)
    fig.show()

In [ ]:
corr = []
for method in ['pearson', 'kendall', 'spearman']:
    temp = df[num_columns].corrwith(df['price'], method=method).rename(method)
    corr.append(temp)
corr_df = pd.DataFrame(corr)
corr_df

In [ ]:
corr_df.T.plot.bar()

In [ ]:
px.bar(corr_df.mean().abs().sort_values(ascending=False),
       color=corr_df.mean().abs().sort_values(ascending=False).index)

In [ ]:
for feature in corr_df.mean().abs().sort_values(ascending=False).index[:4]:
    fig = px.scatter(df, x=feature,y='price', trendline='lowess')
    fig.show()

In [ ]:
corr_mat = df[num_columns].corr()
corr_mat

In [ ]:
corr_mat_one_side = corr_mat.copy()
for i in range(len(corr_mat_one_side)):
    for j in range(i):
        corr_mat_one_side.iat[i, j] = 0
corr_mat_one_side

filt1 = corr_mat_one_side.melt(ignore_index=False)['value'] != 1
filt2 = corr_mat_one_side.melt(ignore_index=False)['value'] != 0
filt = filt1 & filt2
corr_mat_one_side.melt(ignore_index=False)[filt].sort_values('value', ascending=False).head(20)